<a href="https://colab.research.google.com/github/mori8/NLP-Pytorch-practice/blob/main/pytorch_lecture/Chapter_9_Softmax_Classifier_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Exercise 9-2: Build a softmax classifier for Otto Group Product
# https://www.kaggle.com/c/otto-group-product-classification-challenge

import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [3]:
class OttoDataset(Dataset):
  def __init__(self):
    xy = pd.read_csv('/content/drive/MyDrive/NLP-Pytorch/data/otto_train.csv')
    xy = xy.drop(['id'], axis=1)
    self.len = xy.shape[0]
    le = LabelEncoder()
    xy['target'] = le.fit_transform(xy['target'])
    self.y_data = torch.tensor(xy['target'].values, dtype=torch.float32)
    self.x_data = torch.tensor(xy.drop(['target'], axis=1).values, dtype=torch.float32)

  def __getitem__(self, index):
    # return one item on the index
    return self.x_data[index], self.y_data[index]
  
  def __len__(self):
    # return the data length
    return self.len

In [4]:
dataset = OttoDataset()
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

In [5]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.l1 = nn.Linear(93, 520)
    self.l2 = nn.Linear(520, 320)
    self.l3 = nn.Linear(320, 240)
    self.l4 = nn.Linear(240, 120)
    self.l5 = nn.Linear(120, 9)
  
  def forward(self, x):
    x = F.relu(self.l1(x))
    x = F.relu(self.l2(x))
    x = F.relu(self.l3(x))
    x = F.relu(self.l4(x))
    return self.l5(x)

In [6]:
net = Net()
# CrossEntropyLoss의 output은 float, target은 long
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.5)

In [7]:
def train(epoch):
  net.train()
  for batch_idx, d in enumerate(train_set):
    data, target = d
    data, target = Variable(data), Variable(target)
    optimizer.zero_grad()
    output = net(data).reshape(1, 9)
    target = target.long().reshape(1)  
    # print(output.size(), target.size())
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 5000 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx, len(train_set),
          100. * batch_idx / len(train_set), loss.data.item() 
      ))

In [8]:
def test():
  net.eval()
  test_loss = 0
  correct = 0
  for data, target in test_set:
    data, target = Variable(data, volatile=True), Variable(target)
    output = net(data).reshape(1, 9)
    target = target.long().reshape(1)
    test_loss += criterion(output, target).data.item()
    pred = torch.max(output.data, 1)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()
  
  test_loss /= len(test_set.dataset)
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_set),
      100. * correct / len(test_set)
  ))

In [9]:
for epoch in range(1, 5):
  train(epoch)
  test()

Train Epoch: 1 [0/49502 (0%)]	Loss: 2.171056
Train Epoch: 1 [5000/49502 (10%)]	Loss: 0.011307
Train Epoch: 1 [10000/49502 (20%)]	Loss: 0.114568
Train Epoch: 1 [15000/49502 (30%)]	Loss: 0.008966
Train Epoch: 1 [20000/49502 (40%)]	Loss: 1.353779
Train Epoch: 1 [25000/49502 (51%)]	Loss: 0.000000
Train Epoch: 1 [30000/49502 (61%)]	Loss: 0.015084
Train Epoch: 1 [35000/49502 (71%)]	Loss: 0.081543
Train Epoch: 1 [40000/49502 (81%)]	Loss: 0.000005
Train Epoch: 1 [45000/49502 (91%)]	Loss: 0.115196


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  



Test set: Average loss: 0.1341, Accuracy: 9280/12376 (75%)

Train Epoch: 2 [0/49502 (0%)]	Loss: 0.750336
Train Epoch: 2 [5000/49502 (10%)]	Loss: 0.001803
Train Epoch: 2 [10000/49502 (20%)]	Loss: 0.051739
Train Epoch: 2 [15000/49502 (30%)]	Loss: 0.061510
Train Epoch: 2 [20000/49502 (40%)]	Loss: 1.235937
Train Epoch: 2 [25000/49502 (51%)]	Loss: 0.000000
Train Epoch: 2 [30000/49502 (61%)]	Loss: 0.029719
Train Epoch: 2 [35000/49502 (71%)]	Loss: 0.117802
Train Epoch: 2 [40000/49502 (81%)]	Loss: 0.000000
Train Epoch: 2 [45000/49502 (91%)]	Loss: 0.100939

Test set: Average loss: 0.1341, Accuracy: 9307/12376 (75%)

Train Epoch: 3 [0/49502 (0%)]	Loss: 1.262532
Train Epoch: 3 [5000/49502 (10%)]	Loss: 0.003015
Train Epoch: 3 [10000/49502 (20%)]	Loss: 0.021728
Train Epoch: 3 [15000/49502 (30%)]	Loss: 0.059515
Train Epoch: 3 [20000/49502 (40%)]	Loss: 1.182616
Train Epoch: 3 [25000/49502 (51%)]	Loss: 0.000000
Train Epoch: 3 [30000/49502 (61%)]	Loss: 0.008057
Train Epoch: 3 [35000/49502 (71%)]	Loss: